In [1]:
import azureml.core
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.17


In [2]:
import os

In [3]:
from azureml.core import Workspace, Datastore

ws = Workspace.from_config("../config2.json")
print("Workspace {} is loaded".format(ws.name))

Found the config file in: /Users/sleebapaul/AML_MSFT/config2.json
Workspace Auria is loaded


In [4]:
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type, ds.container_name, ds.account_name)

workspaceblobstore AzureBlob azureml-blobstore-963d160e-9bdd-40a8-852c-1fc06dfe7c7a auria5226806917
workspacefilestore AzureFile azureml-filestore-963d160e-9bdd-40a8-852c-1fc06dfe7c7a auria5226806917
videos AzureBlob videos happypathspublic
models AzureBlob styletransfer pipelinedata
breeds AzureBlob azureml-blobstore-315a742f-5c1f-465a-9486-7fd50bbb0012 danielscstoragebhmgfqha
images_datastore AzureBlob sampledata pipelinedata


In [5]:
cts = ws.compute_targets
for ct in cts:
    print(ct)

testing6854
demo-dogbreeds
gpuclusterNCv2
gpuclusterNCv3
cpucluster
v100cluster1
gpucluster
cpu-cluster
gpu-cluster


In [6]:
from azureml.core.compute import AmlCompute

# CPU Cluster
cpu_compute_target = "cpucluster"

try:
    cpu_compute = AmlCompute(ws, cpu_compute_target)
    print("Found existing compute target: {}".format(cpu_compute.name))
except:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1,
                                                                max_nodes = 4)
    cpu_compute = AmlCompute.create(ws, cpu_compute_target, provisioning_config)
    cpu_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    print("Created new compute target: {}".format(cpu_compute.name))

# GPU Cluster
gpu_compute_target = "gpuclusterNCv2"

try:
    gpu_compute = AmlCompute(ws, gpu_compute_target)
    print("Found existing compute target: {}".format(gpu_compute.name))
except:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6S_V2",
                                                                min_nodes = 1,
                                                                max_nodes = 4)
    gpu_compute = AmlCompute.create(ws, gpu_compute_target, provisioning_config)
    gpu_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    print("Created new compute target: {}".format(gpu_compute.name))
    
    
# gpu_compute_target = "gpuclusterNCv3"

# try:
#     gpu_compute = AmlCompute(ws, gpu_compute_target)
#     print("Found existing compute target: {}".format(gpu_compute.name))
# except:
#     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6S_V3",
#                                                                 min_nodes = 1,
#                                                                 max_nodes = 4)
#     gpu_compute = AmlCompute.create(ws, gpu_compute_target, provisioning_config)
#     gpu_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
#     print("Created new compute target: {}".format(gpu_compute.name))

Found existing compute target: cpucluster
Found existing compute target: gpuclusterNCv2


In [7]:
gpu_compute.name

'gpuclusterNCv2'

In [8]:
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData

bs = Datastore.get(ws, datastore_name='workspaceblobstore')
photo_wct = DataReference(
    datastore=bs,
    data_reference_name="photo_wct",
    path_on_datastore="sleebaBlobs/photo_wct.pth",
    mode="download")

fs = Datastore.get(ws, datastore_name='workspacefilestore')
wikiart_emotions = DataReference(
    datastore=fs,
    data_reference_name="WikiArt_Emotions",
    path_on_datastore="WikiArt-Emotions")

output_dir = DataReference(datastore=fs, 
                           data_reference_name= "colored_images",
                           path_on_datastore="colored_images")


                          
# input_image = PipelineParameter(name="input_image")

print("DataReference object created")

DataReference object created


In [9]:
output_dir

$AZUREML_DATAREFERENCE_colored_images

In [19]:
from azureml.train.estimator import Estimator
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core.graph import PipelineParameter

from azureml.core.runconfig import CondaDependencies, RunConfiguration

project_folder = "../NVIDIAFastPhotoStyle/"


script_params = {
    '--content_image_path': "./Auria.png",
    '--model_path': photo_wct,
    '--style_image_path': wikiart_emotions,
    '--output_image_path': output_dir
}

estimator = Estimator(source_directory=project_folder,
                      compute_target=gpu_compute,
                      entry_script='demo.py',
                      script_params=script_params,
                      node_count=1,
                      process_count_per_node=1,
                      
                      pip_packages=[ "http://download.pytorch.org/whl/cu91/torch-0.4.0-cp36-cp36m-linux_x86_64.whl", 
                                    "torchvision==0.2.1", "scikit-umfpack", "setuptools", "numpy","Pillow", "scipy",
                                    "pynvrtc==9.1", "cupy-cuda91"],
                      custom_docker_image = "sleebapaul/cuda_9_1_conda:ubuntu",
                      use_gpu=True)
estimator._estimator_config.environment.python.user_managed_dependencies=False


print("Coloring step created")

Coloring step created


In [20]:
from azureml.core import Experiment

experiment_name = 'Coloring_Exp'
experiment = Experiment(ws, name=experiment_name)

In [21]:
coloring_pipeline_run = experiment.submit(estimator)

print("Pipeline is submitted for execution")

Pipeline is submitted for execution


In [17]:
coloring_pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
Coloring_Exp,Coloring_Exp_1556345487_d2309c57,azureml.scriptrun,Preparing,Link to Azure Portal,Link to Documentation


In [57]:
from azureml.widgets import RunDetails

RunDetails(coloring_pipeline_run).show()

AttributeError: type object 'RunClient' has no attribute 'create'